In [1]:
import cv2
import matplotlib.pyplot as plt
import skimage.segmentation as seg
import skimage.filters as filt
import skimage.morphology as morph
import skimage.draw as draw
from scipy import ndimage
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import cumtrapz
import scipy.ndimage.filters as filters
import os
import numpy as np
import pandas as pd
from plantcv import plantcv as pcv
import scipy.integrate
import time


d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [42]:
def resizeImage(img_path, new_width):
    # Load the image
    img = cv2.imread(img_path)
    # Get the image dimensions
    height, width = img.shape[:2]
    # Calculate the aspect ratio
    ratio = float(new_width) / width
    # Calculate the new height
    new_height = int(height * ratio)
    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    return resized_img

    
def curvature(x, y, xc, yc, r):
    # Shift coordinates to make the center of the circle the origin
    x_shifted = x - xc
    y_shifted = y - yc
    # Calculate distance from each point to the center of the circle
    d = np.sqrt(x_shifted**2 + y_shifted**2)
    # Calculate the curvature only for points inside the circle
    inside_circle = d <= r
    dx_dt = np.gradient(x[inside_circle])
    dy_dt = np.gradient(y[inside_circle])
    d2x_dt2 = np.gradient(dx_dt)
    d2y_dt2 = np.gradient(dy_dt)
    curvature = (dx_dt * d2y_dt2 - d2x_dt2 * dy_dt) / (dx_dt**2 + dy_dt**2)**(3/2)
    # Pad the curvature array with zeros for the points outside the circle
    curvature_padded = np.zeros_like(d)
    curvature_padded[inside_circle] = curvature
    return curvature_padded


def integral_curvature(x, y, xc, yc, r):
    k = curvature(x, y, xc, yc, r)
    return cumtrapz(k, initial=0)


def get_integral_curvature(i,x,y, r):
    # Calculate the integral curvature for a circle centered at (x[i], y[i]) with radius r
    int_curv = integral_curvature(x, y, x[i], y[i], r)[-1]
    return int_curv


def get_area_measure(contour,i,radius, thresh):
    # Approximate the contour with a circle centered at the current point
    circle_mask = np.zeros_like(gray)
    cv2.circle(circle_mask, (contour[i][0][0], contour[i][0][1]), radius, (255, 255, 255), -1)
    intersection_mask = cv2.bitwise_and(circle_mask, thresh)
    intersection_area = np.sum(intersection_mask) / 255
    circle_area = np.pi * radius ** 2
    intersection_fraction = intersection_area / circle_area
    # Compute the curvature value for the current point
    return intersection_fraction


def get_curvature_arc_length(contour,i,x,y,radius,circle_perimeters):
    curvature_arr = []
    # Approximate the contour with a circle centered at the current point
    (cx, cy) = x[i], y[i]
    # Calculate the length of the part of the circle's circumference that is inside the object
    chord_length = 2 * np.sqrt(radius**2 - ((radius**2-(x[i]-cx)**2-(y[i]-cy)**2))/4)
    arc_length = 2 * np.arcsin(chord_length / (2 * radius))
    circle_perimeters.append(arc_length)
    if len(circle_perimeters) > 0:
        # Calculate the average perimeter of the fitted circles
        circle_perimeter = np.mean(circle_perimeters)
    else:
        # Use the radius as an estimate for the perimeter
        circle_perimeter = 2 * np.pi * radius

    # Calculate perimeter of the contour
    contour_perimeter = cv2.arcLength(contour, True)
    # Calculate the fraction of the circle's perimeter contained inside the object
    fraction = circle_perimeter / contour_perimeter
    arc_len = fraction * arc_length
    return arc_len


def get_curvatures(contours, radius_arr,thresh):
    curvature_arr=[]
    # Loop over each contour
    for contour in contours:
        curr = []
        # Precompute the coordinates of all points in the contour
        x, y = contour[:, 0, 0], contour[:, 0, 1]
        # Compute the maximum and minimum x and y coordinates of the contour
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)
        # Compute the thresholded image for the current contour
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, [contour], 0, (255, 255, 255), -1)
        thresh = cv2.bitwise_and(gray, mask)
        # Compute the area measure for each point in the contour
        for radius in radius_arr:
            circle_perimeters = []
            arc_length = 0  # Initialize arc_length to 0
            curr = []
            for i in range(len(contour)):
                curr_area = get_area_measure(contour,i,radius,thresh)
                print(curr_area)
                # curr_arc = get_curvature_arc_length(contour,i,x,y,radius,circle_perimeters)
                # curr_int = get_integral_curvature(i,x,y, radius)
                curr.append(curr_area) # remove
                #curr.append(curr_arc)
                #curr.append(curr_int) # add
            curvature_arr.append(curr)

    return curvature_arr
            #print( f'area={curr_area} arc ={curr_arc} curr int = {curr_int}')

In [43]:
radius_arr = np.arange(10, 131, 5)

In [45]:
# specify path to the folder containing image files
path = "../../leafsnap/leafsnap-dataset/dataset/segmented/field"
# path = "../../leafsnap/leafsnap-dataset/dataset/segmented/field" D:\University\4-2\Thesis\leafsnap\leafsnap-dataset\dataset\segmented\field

counter = 0

# initialize empty lists for features and labels
features = []
labels = []

# loop through all subfolders in the path
for foldername in os.listdir(path):

    if counter == 1:
        break
    
    folderpath = os.path.join(path, foldername)
    if not os.path.isdir(folderpath):
        continue
        
    # loop through all image files in the subfolder
    for filename in os.listdir(folderpath):

        filepath = os.path.join(folderpath, filename)
        if not os.path.isfile(filepath):
            continue
        
        # try:
            
        start_time = time.time()
        # Load the image
        img = cv2.imread(filepath)
        # set the new size
        new_size = (250, 250)
        # resize the image
        img = cv2.resize(img, new_size)
        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        # Apply a Gaussian blur to the image to reduce noise
        blur = cv2.GaussianBlur(gray, (5,5), 0)
        # Apply Canny edge detection to get the edges
        edges = cv2.Canny(blur, 100, 200)
        # Find the contours in the image
        contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # Merge all the contours
        merged_contours = []
        for contour in contours:
            merged_contours += contour.tolist()
        oc = np.array([merged_contours])
        print(oc)
        currvature_arr = get_curvatures(oc,radius_arr,thresh)
        print(currvature_arr)
        curvature_arr = np.array(currvature_arr[0])
        x = np.arange(curvature_arr.shape[1]) # x-axis is the number of contour points
        y = np.arange(curvature_arr.shape[0]) # y-axis is the different scales
        X, Y = np.meshgrid(x, y)
        hist_values = []
        for i in range(curvature_arr.shape[0]):
            counts, bin_edges = np.histogram(curvature_arr[i], bins=21) # adjust the number of bins as needed
            hist_values.append(counts)
        data = np.array([np.array(hist) for hist in hist_values])
        X2 = data.flatten().tolist()
        print(X2)
        features.append(X2)
        labels.append(foldername)
        end_time = time.time()
        total_time = end_time - start_time
        print(f"Total run time: {total_time} seconds")

        # except:
        #     pass

    counter += 1
        
    print(f"Finished plant: {foldername}")
    
# convert features and labels to numpy arrays
features = features
labels = np.array(labels)

# save features and labels to a pandas dataframe and export to CSV file
data = pd.DataFrame({"hist_values" : features})
data["plant"] = labels
data.to_csv("lf_data.csv", index=False)

[[[[123 102]]

  [[122 103]]

  [[122 105]]

  [[121 106]]

  [[121 109]]

  [[119 111]]

  [[119 112]]

  [[118 113]]

  [[118 114]]

  [[117 115]]

  [[117 116]]

  [[116 117]]

  [[116 118]]

  [[115 119]]

  [[115 120]]

  [[113 122]]

  [[112 122]]

  [[111 123]]

  [[111 125]]

  [[110 126]]

  [[110 131]]

  [[111 132]]

  [[111 135]]

  [[112 136]]

  [[112 138]]

  [[111 139]]

  [[110 139]]

  [[109 140]]

  [[ 94 140]]

  [[ 93 141]]

  [[ 90 141]]

  [[ 91 142]]

  [[ 91 144]]

  [[ 98 151]]

  [[ 98 152]]

  [[102 156]]

  [[103 156]]

  [[105 158]]

  [[106 158]]

  [[109 161]]

  [[109 163]]

  [[110 164]]

  [[111 164]]

  [[112 165]]

  [[123 165]]

  [[124 164]]

  [[127 164]]

  [[128 163]]

  [[129 164]]

  [[130 164]]

  [[131 163]]

  [[132 164]]

  [[133 163]]

  [[134 163]]

  [[135 162]]

  [[136 162]]

  [[137 163]]

  [[154 163]]

  [[155 162]]

  [[158 162]]

  [[158 160]]

  [[162 156]]

  [[164 156]]

  [[166 154]]

  [[167 154]]

  [[167 153]]

  [[170 15

IndexError: tuple index out of range

In [20]:
import cv2
import numpy as np
from plantcv import plantcv as pcv
import matplotlib.pyplot as plt

# Load the image
img = cv2.imread('../../leafsnap/leafsnap-dataset/dataset/segmented/lab_/1/acer_campestre/ny1079-01-1.png')

# Preprocess the image using PlantCV
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
binary_img = pcv.threshold.binary(gray_img, 100, 255, 'light')

filled_img = pcv.fill_holes(bin_img=binary_img)

# Get the contours using PlantCV
contours, _ = pcv.find_objects(filled_img, binary_img)

# Set the radius and number of bins
radius = 25
nbins = 21

# Initialize the curvature and area measure lists
curvature_arr = []
area_vals = []

# Loop through the contours
for i, contour in enumerate(contours):
    
    # Fill in the contour to remove holes in the shape
    filled_contour = np.zeros_like(binary_img)
    cv2.drawContours(filled_contour, contours, i, color=255, thickness=-1)
    
    # Resize the filled-in image to a common area for scale invariance
    resized_img = cv2.resize(filled_contour, (200, 200))
    
    # Compute the curvature of the contour
    curvature_image = pcv.morphology.compute_curvature(resized_img, radius)
    
    # Compute the area measure
    area_measure = pcv.morphology.area_measure(resized_img, contour, radius)
    area_vals.append(area_measure)
    
    # Compute the histograms of curvature values at each scale
    for j in range(radius):
        curvature_hist, _ = np.histogram(curvature_image[:, j], bins=nbins, range=(-1, 1))
        curvature_vals.append(curvature_hist)
        
# Concatenate the histograms to form the HoCS feature
feature = np.concatenate(curvature_vals).ravel()

# Display the histogram over curvature values
plt.hist(feature, bins=nbins)
plt.show()

AttributeError: module 'plantcv.plantcv.morphology' has no attribute 'compute_curvature'